In [1]:
import numpy as np
import matplotlib.pyplot as plt
from celluloid import Camera

def init_params(N,v0):
    spp_coords=np.random.uniform(0,1,(N,2)) # spp_coords[i]==(x,y) position of spp i
    spp_thetas=np.random.uniform(-np.pi,np.pi,(N,1))
    spp_vel=np.empty((N,2))
    spp_vel[:,0]=v0*np.cos(spp_thetas.T)
    spp_vel[:,1]=v0*np.sin(spp_thetas.T)
    return spp_coords,spp_vel

def get_neighbors(pos,r):
    N=len(pos)
    nb=[[] for i in range(N)]
    for i in range(N):
        for j in range(N):
            if(i==j): 
                continue
            dx=pos[i][0]-pos[j][0]
            if dx<-.5: dx=+1
            elif dx>=.5: dx+=-1
            dy=pos[i][1]-pos[j][1]
            if dy<-.5: dy=+1
            elif dy>=.5: dy+=-1
        
            dist=np.sqrt(dx**2+dy**2)
            
            if(dist>r):
                continue
            nb[i].append(j)
    return nb

def avg_vec(vel,nb):
    N=len(nb)
    avg_vec=np.zeros((N,2))
    for i in range(N):
        if len(nb[i])==0: continue
        sumvec=np.array([0.,0.])
        for nbs in nb[i]:
            sumvec+=vel[nbs]
        avg_vec[i]=sumvec*(1/len(nb[i]))
    return avg_vec

        
        
def vicsek():
    N=int(input('Number of Self-Propelled Particles: N='))
    v0=float(input('Velocity v0 (fixed for all Particles): v0='))
    eta=float(input('Noise: eta='))
    r=float(input('interaction Range: R='))
    dt=float(input('timestep: dt='))
    iterations=int(input('iterations:'))
    pos,vel=init_params(N,v0)
    nb=get_neighbors(pos,r)
    t,T=0.,dt*iterations
    fig=plt.figure()
    plt.xlim(0,1)
    plt.ylim(0,1)
    camera = Camera(fig)
    while t<T:
        ksi=eta*np.random.uniform(-.5,.5,(N,2))
        avg=avg_vec(vel,nb)
        plt.plot(pos[:,0],pos[:,1],ls='',marker='+',color='blue')
        camera.snap()
        pos=pos+(avg+ksi)*dt
        vel=avg+ksi
        pos[pos<0]+=1
        pos[pos>=1]+=-1
        nb=get_neighbors(pos,r)
        t=t+dt
    anim = camera.animate(blit=True)
    anim.save('VicsekSim-for-'+str(N)+'-SPPs-v0='+str(v0)+'-eta(noise)='+str(eta)+'-R='+str(r)+'.mp4')
